In [6]:
import os
from simple_salesforce import Salesforce, SalesforceLogin
from simple_salesforce.exceptions import SalesforceAuthenticationFailed, SalesforceGeneralError
import requests
import csv
from datetime import datetime
from google.oauth2.credentials import Credentials 
from googleapiclient.discovery import build

# Configura tus credenciales aquí
USERNAME = 'yonatan.osorio@openenglish.com'
PASSWORD = 'Yonatan26++'
SECURITY_TOKEN = 'ho7zyIcpDbybcP1G7tLbor6sU'  # si aplica
DOMAIN = 'login'  # usa 'test' para sandbox

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CREDENTIALS_FILE = 'panels-id.json'
TOKEN_FILE = 'token.json'

# ID del reporte que quieres descargar
REPORTS = {
    'Lead OE Adults': '00O3r000007Y8AX',
    'Lead OE Junior': '00O3r000007Y8hH',
    'Lead OE España': '00OKV000008PVBz',
    'Lead OE Venezuela': '00O3r000007Y8bs',
    'Lead OE Turkey': '00O3r000007Y959',
    'Lead OE Brasil': '00O3r000007Y8dU'
}

# Variables definidas fuera de las funciones
sheet_id = '1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80'  # ID de la hoja de cálculo
hoja = 'Leads'  # Nombre de la pestaña en la hoja de cálculo
csv_file = 'Descargas/Lead OE Venezuela.csv'  # Ruta del archivo CSV
rango_limpiar = 'A2:K10000'  # El rango que se borrará (debe coincidir con el tamaño de los datos)
rango_inicial = 'A2'  # El rango donde se pegarán los nuevos datos

archivos_sheets = {
    "Lead OE Adults.csv": {
        "sheet_id": "1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4",
        "hoja": "Leads"
    },
    "Lead OE Junior.csv": {
        "sheet_id": "1wXbaCBJrC_TxXgaGo-14NVAkS3uzRo_a6oWUP5ux7KU",
        "hoja": "Leads"
    },
    "Lead OE Turkey.csv": {
        "sheet_id": "1bA1GEhfufFq9_iLczSMzhFGFbKZoHwe7K4g_9g_vCsU",
        "hoja": "Leads"
    },
    "Lead OE Brasil.csv": {
        "sheet_id": "14t9cSHrJ1VexbOtm1ybmYPg9iiewuAK2j1YEPP4rB4E",
        "hoja": "Leads"
    },
    "Lead OE España.csv": {
        "sheet_id": "1LxXDXw8B4xP0Y6_PVexnaXpnBUUPL7kNP9IJiULBo6o",
        "hoja": "Leads"
    },
    "Lead OE Venezuela.csv": {
        "sheet_id": "1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80",
        "hoja": "Leads"
    },
}

FOLDER_PATH = 'Descargas'
LOG_FOLDER = os.path.join(FOLDER_PATH, 'Logs')
os.makedirs(FOLDER_PATH, exist_ok=True)
os.makedirs(LOG_FOLDER, exist_ok=True)

# Lista para reportes omitidos por restricción de hora
omitidos_por_tiempo = []

# Conexión a Salesforce
try:
    session_id, instance = SalesforceLogin(
        username=USERNAME,
        password=PASSWORD,
        security_token=SECURITY_TOKEN,
        domain=DOMAIN
    )
    sf = Salesforce(instance=instance, session_id=session_id)
    print("✅ Conexión exitosa a Salesforce")

except SalesforceAuthenticationFailed as e:
    print(f"❌ Error de autenticación: {e}")
    
except Exception as e:
    print(f"❌ Error general al autenticar: {e}")

# Función para borrar y reemplazar los datos de la hoja de cálculo de Google
def borrar_y_reemplazar_datos():
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    service = build('sheets', 'v4', credentials=creds)

    # Limpiar el rango especificado
    service.spreadsheets().values().clear(
        spreadsheetId=sheet_id,
        range=f"{hoja}!{rango_limpiar}"
    ).execute()
    
    # Leer los datos del archivo CSV
    with open(csv_file, mode='r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        values = list(reader)

    # Pegar los nuevos datos en el rango especificado
    service.spreadsheets().values().update(
        spreadsheetId=sheet_id,
        range=f"{hoja}!{rango_inicial}",
        valueInputOption="RAW",
        body={"values": values}
    ).execute()

    print(f"✅ Datos reemplazados en la hoja '{hoja}' desde el rango {rango_inicial} (rango limpiado: {rango_limpiar})")


# Función para actualizar la hoja desde CSV
def actualizar_hoja_desde_csv():
    borrar_y_reemplazar_datos()

# Llamada a la función para actualizar la hoja
actualizar_hoja_desde_csv()

# Continuación de tu código de descarga de reportes y subida a Google Sheets...

# Subir archivos manuales a Google Sheets
try:
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    for nombre_archivo, datos in archivos_sheets.items():
        if nombre_archivo in omitidos_por_tiempo:
            print(f"⏩ Se omite la subida de '{nombre_archivo}' porque fue omitido por tiempo.")
            continue

        ruta_archivo = os.path.join(FOLDER_PATH, nombre_archivo)
        if not os.path.exists(ruta_archivo):
            print(f"⚠️ Archivo no encontrado: {ruta_archivo}")
            continue

        with open(ruta_archivo, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            values = list(reader)

        sheet_id = datos['sheet_id']
        hoja = datos['hoja']
        rango = f"{hoja}!A1"
        service.spreadsheets().values().update(
            spreadsheetId=sheet_id,
            range=rango,
            valueInputOption="RAW",
            body={"values": values}
        ).execute()
        print(f"✅ Subido '{nombre_archivo}' a hoja '{hoja}' en Google Sheets")

except Exception as e:
    print(f"❌ Error al subir a Google Sheets: {e}")


✅ Conexión exitosa a Salesforce
✅ Datos reemplazados en la hoja 'Leads' desde el rango A2 (rango limpiado: A2:K10000)
✅ Subido 'Lead OE Adults.csv' a hoja 'Leads' en Google Sheets
✅ Subido 'Lead OE Junior.csv' a hoja 'Leads' en Google Sheets
✅ Subido 'Lead OE Turkey.csv' a hoja 'Leads' en Google Sheets
✅ Subido 'Lead OE Brasil.csv' a hoja 'Leads' en Google Sheets
✅ Subido 'Lead OE España.csv' a hoja 'Leads' en Google Sheets
✅ Subido 'Lead OE Venezuela.csv' a hoja 'Leads' en Google Sheets


In [29]:
from datetime import datetime 

current_time = datetime.now().time()
limit_time = datetime.strptime("18:00", "%H:%M").time()


print(current_time)
print(limit_time)

18:59:21.246445
18:00:00
